<strong><b><font size="5">Time Series Analysis and Forecasting in Python</font></b></strong>

In [ ]:
!pip3 install statsmodels
!pip3 install pmdarima

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
plt.style.use('fivethirtyeight')
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

# Importing Data
- Dataset: Monthly Milk Production 

In [ ]:
df = pd.read_csv('./monthly_milk_production.csv', sep=',', parse_dates=['Date'], index_col='Date')

df.plot(figsize=(12,5))
plt.title('Monthly Milk Production')
plt.show()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.shape

# Data Inspection

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=False, sharey=False, figsize=(12,5))
df.hist(ax=ax1)
df.plot(kind='kde', ax=ax2)
plt.show();

__The distribution is not perfectly Gaussian normal distribution and it is left shifted.__

## Time Series Decomposition

In [ ]:
decomposition = seasonal_decompose(df['Production'], period=12, model='additive')
plt.rcParams['figure.figsize'] = 12, 5
decomposition.plot()
plt.show();

## Seasonal Component

In [ ]:
df['year'] = df.index.year
df['month'] = df.index.month
df_pivot = pd.pivot_table(df, values='Production', index='month', columns='year', aggfunc='mean')
df_pivot.plot(figsize=(12,8))
plt.legend().remove()
plt.xlabel('Month')
plt.ylabel('Milk Production')
plt.show()

In [ ]:
df.drop(['year', 'month'], axis=1, inplace=True)

# Model Identification
__Check for Stationarity__

## ACF and PACF plots

__ACF and PACF plots: If the time series is stationary, the ACF/PACF plots will show a quick cut off after a small number of lags.__

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(14,6), sharex=False, sharey=False)
ax1 = plot_acf(df, lags=50, ax=ax1)
ax2 = plot_pacf(df, lags=50, ax=ax2)
plt.show()

In [ ]:
#Determing rolling statistics
rolmean = pd.Series(df['Production']).rolling(window=12).mean()
rolstd = pd.Series(df['Production']).rolling(window=12).std()

#Plot rolling statistics:
orig = plt.plot(df['Production'], color='blue',label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

## Statistical Tests

__Augmented Dickey Fuller Test__
If the p-value is lower than the threshold value (5% or 1%), we reject the null hypothesis and time series is stationary. If the p-value is higher than the threshold, we fail to reject the null hypothesis and time series is non-stationary.

In [ ]:
print ('Results of Dickey-Fuller Test:')
dftest = adfuller(df['Production'])

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

## Making Time Series Stationary

In [ ]:
df_diff = df.diff().diff(12)

In [ ]:
df_diff.dropna(inplace=True)

In [ ]:
print ('Results of Dickey-Fuller Test:')
dftest = adfuller(df_diff['Production'])

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

In [ ]:
plt.plot(df_diff['Production'])
plt.title('Monthly Milk Production')
plt.savefig('diffplot')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(14,6), sharex=False, sharey=False)
ax1 = plot_acf(df_diff, lags=50, ax=ax1)
ax2 = plot_pacf(df_diff, lags=50, ax=ax2)
plt.savefig('acfpacf2')
plt.show()

# Model Parameter Estimation

In [ ]:
import pmdarima as pm
model = pm.auto_arima(df['Production'], d=1, D=1,
                      seasonal=True, m=12, trend='c', 
                      start_p=0, start_q=0, max_order=6, test='adf', stepwise=True, trace=True)


In [ ]:
model.summary()

In [ ]:
#divide into train and validation set
train = df[:int(0.85*(len(df)))]
test = df[int(0.85*(len(df))):]

#plotting the data
train['Production'].plot()
test['Production'].plot()

In [ ]:
model = SARIMAX(train['Production'],order=(1,1,0),seasonal_order=(0,1,1,12))
results = model.fit()
results.summary()

# Statistical Model Checking

__Model Diagnostics__

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.savefig('modeldiagnostics')
plt.show()

In [ ]:
forecast_object = results.get_forecast(steps=len(test))

mean = forecast_object.predicted_mean

conf_int = forecast_object.conf_int()

dates = mean.index

In [ ]:
start=len(train)
end=len(train)+len(test)-1
predictions = results.predict(start=start, end=end, dynamic=False, typ='levels').rename('SARIMA(1,1,0)(0,1,1,12) Predictions')

In [ ]:
# Compare predictions to expected values
for i in range(len(predictions)):
    print(f"predicted={predictions[i]:<11.10}, expected={test['Production'][i]}")

In [ ]:
# Plot predictions against known values
title = 'Monthly Milk Production'
ax = test['Production'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
#ax.set(xlabel=xlabel, ylabel=ylabel);

In [ ]:
r2_score(test['Production'], predictions)

In [ ]:
evaluation_results = pd.DataFrame({'r2_score': r2_score(test['Production'], predictions)}, index=[0])
evaluation_results['mean_absolute_error'] = mean_absolute_error(test['Production'], predictions)
evaluation_results['mean_squared_error'] = mean_squared_error(test['Production'], predictions)
evaluation_results['mean_absolute_percentage_error'] = np.mean(np.abs(predictions - test['Production'])/np.abs(test['Production']))*100 

evaluation_results

# Future Forecasts

In [ ]:
pred_f = results.get_forecast(steps=60)
pred_ci = pred_f.conf_int()
ax = df.plot(label='Production', figsize=(14, 7))
pred_f.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Monthly Milk Production')
plt.legend()
plt.show()